In [1]:
import os
import openai
import jsonlines
import tiktoken 
import datetime

openai.api_key = os.getenv("OPENAI_API_KEY")

In [8]:

#upload the training data.
openai.File.create(file=open("data/train_data.jsonl", "rb"),purpose='fine-tune')

<File file id=file-FyLSOqxGZ8mdoPgmnwnK47H4 at 0x117d3e630> JSON: {
  "object": "file",
  "id": "file-FyLSOqxGZ8mdoPgmnwnK47H4",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 55048,
  "created_at": 1699523164,
  "status": "processed",
  "status_details": null
}

In [9]:
# check the uploaded files
response = openai.File.list()

for file in response['data']:
    created_at = datetime.datetime.fromtimestamp(file['created_at']).strftime('%Y-%m-%d %H:%M:%S')
    print(f"File ID: {file['id']}, Filename: {file['filename']}, Created at: {created_at}")

File ID: file-FyLSOqxGZ8mdoPgmnwnK47H4, Filename: file, Created at: 2023-11-09 17:46:04
File ID: file-b2d8Mi3bsVmQRoRSRRiN7X4V, Filename: step_metrics.csv, Created at: 2023-11-07 19:34:10
File ID: file-Feei9Ga8aUVYwCIpnBwol7CO, Filename: example&document&definition.jsonl, Created at: 2023-11-07 17:33:00
File ID: file-Qpr8N0HOkWbEllVh49FXD799, Filename: step_metrics.csv, Created at: 2023-11-05 23:44:41
File ID: file-rOQcTlH8G9UrSUeO0Dpgozj1, Filename: file, Created at: 2023-11-05 23:39:52
File ID: file-a8vBiTckJ4qISTo2H12JQpA8, Filename: file, Created at: 2023-11-05 23:18:23
File ID: file-n9Hp2ewS9OdaYgWqAlF35Z82, Filename: data/data_call_prepared.jsonl, Created at: 2023-11-05 22:58:04
File ID: file-zjBUwBfIPCMdlSiDaNT2ZmA5, Filename: compiled_results.csv, Created at: 2023-11-05 22:41:51
File ID: file-3A05X4q2InFZ0A0GxMH5hodh, Filename: data/data_call_prepared.jsonl, Created at: 2023-11-05 22:39:30
File ID: file-Vjq5XMXfB0Lv0hS0thIzWi4Q, Filename: data/data_call_prepared.jsonl, Created 

In [10]:
# create finetuning job
openai.FineTuningJob.create(training_file="file-FyLSOqxGZ8mdoPgmnwnK47H4", model="gpt-3.5-turbo")

<FineTuningJob fine_tuning.job id=ftjob-4bp5fhUln078FDrHsqaOXIGD at 0x117a1c3b0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-4bp5fhUln078FDrHsqaOXIGD",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1699523198,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-MWtct4ruIBsIACK6VTRbJmRI",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-FyLSOqxGZ8mdoPgmnwnK47H4",
  "hyperparameters": {
    "n_epochs": "auto",
    "batch_size": "auto",
    "learning_rate_multiplier": "auto"
  },
  "trained_tokens": null,
  "error": null
}

In [13]:
# List 10 fine-tuning jobs
# openai.FineTuningJob.list(limit=10)

# Retrieve the state of a fine-tune
# openai.FineTuningJob.retrieve("ftjob-Kpmv51HPmG5n9eC2MHZoqZU3")
openai.FineTuningJob.retrieve("ftjob-4bp5fhUln078FDrHsqaOXIGD")

# Cancel a job
# openai.FineTuningJob.cancel("ftjob-Kpmv51HPmG5n9eC2MHZoqZU3")

# List up to 10 events from a fine-tuning job
# openai.FineTuningJob.list_events(id="ftjob-Kpmv51HPmG5n9eC2MHZoqZU3", limit=10)

# Delete a fine-tuned model (must be an owner of the org the model was created in)
# openai.Model.delete("ft:gpt-3.5-turbo:acemeco:suffix:abc123")

<FineTuningJob fine_tuning.job id=ftjob-4bp5fhUln078FDrHsqaOXIGD at 0x106d0dfd0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-4bp5fhUln078FDrHsqaOXIGD",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1699523198,
  "finished_at": 1699525627,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::8IwXcIFI",
  "organization_id": "org-MWtct4ruIBsIACK6VTRbJmRI",
  "result_files": [
    "file-IdvSoNnlQKcgJqO0uKKRBnwJ"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-FyLSOqxGZ8mdoPgmnwnK47H4",
  "hyperparameters": {
    "n_epochs": 9,
    "batch_size": 1,
    "learning_rate_multiplier": 2
  },
  "trained_tokens": 102942,
  "error": null
}

In [14]:
# get completion of gpt3.5
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[{"role": "system", "content": " As a helpful assistant, your task is to write an impressive personal statement for a student who is applying for a study program. You will be provided with a title, requirements, and the user's experience to base your essay on. Your goal is to showcase the student's good character in a polished and compelling manner. To accurately reflect the author's point of view, you should use a first-person perspective and incorporate the word \"I\" throughout the essay."}, {"role": "user", "content": "Title: Safeguarding Memories. Requirement : Common Application: Topic of Your Choice. Experience: I used to walk with my grandfather as a child, buying sweet buns from a cart. But as he developed Alzheimer's, he forgot who I was. This inspired me to start the War Stories Project, where we collect and preserve the memories of World War II witnesses. Now, as the coordinator, I interview survivors and manage the project. It's important to me because with each passing year, we lose more of these valuable witnesses. Remembering our history is crucial for our future."}]
)
print(completion.choices[0].message)

{
  "role": "assistant",
  "content": "Personal Statement: Safeguarding Memories\n\nIn a world that seems to be moving at an ever-increasing pace, memories have become precious treasures, cherished fragments of our past that make us who we are. It is through these memories that we connect with our loved ones, understand our history, and shape our future. For as long as I can remember, preserving memories has been at the core of who I am. This passion was ignited by a personal experience that shaped my purpose and drove me to embark on a meaningful journey - the War Stories Project.\n\nAs a young child, I would walk hand in hand with my beloved grandfather, eagerly following the aroma of freshly baked sweet buns from a nearby cart. Those walks were moments of pure joy, etched in my heart forever. However, as the years passed by, a shadow began to fall over these cherished memories. My grandfather was diagnosed with Alzheimer's, and slowly but surely, he began to forget who I was. It was

In [15]:
# get completion of our fine-tuned model
completion = openai.ChatCompletion.create(
  model="ft:gpt-3.5-turbo-0613:personal::8IwXcIFI",
  messages=[{"role": "system", "content": " As a helpful assistant, your task is to write an impressive personal statement for a student who is applying for a study program. You will be provided with a title, requirements, and the user's experience to base your essay on. Your goal is to showcase the student's good character in a polished and compelling manner. To accurately reflect the author's point of view, you should use a first-person perspective and incorporate the word \"I\" throughout the essay."}, {"role": "user", "content": "Title: Safeguarding Memories. Requirement : Common Application: Topic of Your Choice. Experience: I used to walk with my grandfather as a child, buying sweet buns from a cart. But as he developed Alzheimer's, he forgot who I was. This inspired me to start the War Stories Project, where we collect and preserve the memories of World War II witnesses. Now, as the coordinator, I interview survivors and manage the project. It's important to me because with each passing year, we lose more of these valuable witnesses. Remembering our history is crucial for our future."}]
)
print(completion.choices[0].message)

{
  "role": "assistant",
  "content": "Safeguarding Memories\n\nI used to walk downtown with my grandfather. As we strolled burgeoning flower beds and past old oaks, we bought sweet buns from the bakery cart. I vividly remember the toasty-sweet smell that wafted through the dusky air as my grandfather and I waited our turn. However, my grandfather doesn't.\nAt the age of five years old, I understood Alzheimer's in only the most simplistic emotions. \"Why?\" I asked my mother, \"can't Grandpa remember what I did? Why can't I remind him?\" It broke my child's heart when my mother sadly whispered, \"He can't, dear.\"For an instant, I think, I glimpsed the suffering of the disease that slowly sucked away the person I once knew as my grandfather. Slowly, he no longer recognized who I was.\nIt is not only my grandfather that this is happening to. Since 1992, about 20 million Alzheimer's patients have been reported. It is estimated that a new case is diagnosed about every 72 seconds.This cris